# Objectives
Use the `all-MiniLM-L6-v2` model to generate embeddings<br>
Leverage a LLM for Q&A tasks<br>
Add document upload functionality <br>
Employ `FAISS` to retrieve the most semantically similar sentences

## Setup
### Install the required libraries

In [2]:
%%capture output
%pip install -U sentence-transformers
%pip install ollama
%pip install pdfplumber
%pip install python-docx  
%pip install nltk
%pip install ipywidgets

In [3]:
if 'ERROR' in output.stdout or 'Failure' in output.stderr:
    print('One or more of the package installations failed')
    if output.stderr.strip():
        print("Error details:\n", output.stderr.strip())
    else:
        print("Error details:\n", output.stdout.strip())
else:
    print('All packages installed successfully')


All packages installed successfully


### Import the required libraries

In [1]:
# Contains the model to generate sentence embeddings
from sentence_transformers import SentenceTransformer

import numpy as np

# THe local LLM
import ollama

# For in-memory reading of files
from io import BytesIO 

# To read and extract text from various document types
import pdfplumber 
import docx 

# To divide text into individual sentences
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk import sent_tokenize 

# For the File upload functionality
import ipywidgets as widgets
from IPython.display import display

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\bhargav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\bhargav\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
import faiss
print(faiss.__version__)

1.13.2


### A base dataset

In [3]:
sentences = [
    "Vigyanlabs is an innovation-driven organization focused on building Intelligent Power Management and Self-Care products to promote green computing and save money for global enterprises.",
    "Founded in 2008 by Srinivas Varadarajan and Srivatsa Krishnaswamy, Vigyanlabs has attracted top talent to create new intellectual property and deliver sustainable technology solutions.",
    "The mission of Vigyanlabs is to harness the power of emerging technologies to better manage energy-consuming devices and significantly reduce their overall carbon footprint.",
    "Vigyanlabs holds over 15 granted US patents related to energy efficiency, data center optimization, and sustainable digital transformation across various enterprise IT infrastructures.",
    "Headquartered in Mysuru, India, Vigyanlabs operates globally with a presence in major cities like Bangalore, Mumbai, New York, and California to support its international clients.",
    "The company is recognized as a Microsoft Accelerator incubated startup and has won prestigious accolades like the NASSCOM AI Game Changer and Super Startups Asia awards.",
    "Vigyanlabs focuses on 'Profitable Sustainability,' a philosophy that ensures environmental initiatives also drive capital and operational cost savings for the businesses that implement them.",
    "The leadership team at Vigyanlabs brings decades of experience from global technology giants such as HP, Dell, Apple, and various key Indian government technology consulting projects.",
    "Vigyanlabs played a crucial role in shaping the architecture for Aadhaar 1.0 and has provided strategic technology consulting to the Income Tax Department and India Post.",
    "With a valuation of approximately 158Cr, Vigyanlabs continues to lead the deep-tech sector by integrating AI-powered sustainability into private and public cloud platforms.",
    "The flagship product IPM+ is a modular suite covering asset management, patch management, security compliance, endpoint backup, and comprehensive employee productivity monitoring for large organizations.",
    "IPM+ for Endpoints has seen massive adoption with a deployment base of more than 6 million devices across over 26,000 corporate locations worldwide.",
    "The standout feature of IPM+ is the US-patented AI PowerMind Engine, which drives energy savings of up to 50 percent per endpoint in real-world deployments.",
    "Major financial institutions like the State Bank of India and ICICI Lombard rely on the IPM+ suite to manage and optimize their vast device ecosystems.",
    "IPM+ provides a single pane of glass for enterprises to manage all endpoints, including desktops, laptops, smartphones, and tablets through one unified interface.",
    "The IPM+ agent is designed with an ultra-light footprint, consuming less than 3 percent of CPU resources to ensure zero performance degradation for end users.",
    "Vigyanlabs' IPM+ solutions have collectively saved more than 2,000 GWH of energy, demonstrating a significant impact on global sustainability and corporate social responsibility.",
    "The software includes a certified 'Software Energy Meter' that tracks energy impact with over 90 percent accuracy, as verified by independent global third-party auditors.",
    "IPM+ allows administrators to enforce over 500 custom policies in real-time, ensuring that security and operational standards are maintained across the entire organization.",
    "The platform is supported in over 50 countries, providing localized support for diverse IT environments and helping companies reach their global Net-Zero emission targets.",
    "IPM+ uses patented application sensing technology to optimize power consumption by detecting which software is active and adjusting hardware states accordingly without user intervention.",
    "The PowerMind AI platform non-intrusively reduces energy consumption across IT infrastructure by managing CPU throttling, GPU performance, and core parking in real-time.",
    "By lowering heat output from devices, IPM+ helps reduce HVAC cooling costs and UPS capacity requirements by up to 30 percent in office environments.",
    "Users can extend the battery life of portable devices like laptops and tablets by up to 30 percent using the specialized IPM+ energy assistant modules.",
    "The solution provides granular control over individual hardware components, including fan speeds, thermal management, and port-level power control to minimize wasted electricity.",
    "IPM+ provides real-time energy audits and analytics, allowing managers to see exactly how much CO2 emissions have been sequestered through their green computing initiatives.",
    "The system's adaptive AI/ML power policies are optimized for both Windows and Linux operating systems, ensuring broad compatibility across hybrid enterprise IT fleets.",
    "Automated power states from S0 to S5 allow devices to transition into low-power modes during periods of inactivity without disrupting background tasks or scheduled updates.",
    "The IPM+ PowerMind engine is capable of scaling to manage over 250,000 endpoints simultaneously while maintaining fine-grained control over individual user power profiles.",
    "Strategic power management through IPM+ has saved customers like the State Bank of India over 44 Crore Rupees in annual electricity costs across their branches.",
    "Vigyanlabs' UEM solution provides a comprehensive management stack that integrates asset tracking, patch deployment, and security enforcement into a single, lightweight software agent.",
    "The platform offers automated agent-less network discovery to identify all hardware assets, software licenses, and connected peripherals across the entire corporate network.",
    "IPM+ Asset Management provides life cycle and cost management for IT hardware, helping organizations optimize their capital expenditure by identifying under-utilized or obsolete equipment.",
    "The system tracks over 200 parameters per endpoint, providing deep insights into device health, performance trends, and potential hardware vulnerabilities before they cause downtime.",
    "With over 450 built-in dashboards, administrators can monitor security posture, system health, and productivity metrics through a highly customizable and visual analytics interface.",
    "The UEM suite includes a 'Self-Care' module that allows endpoints to automatically remediate common issues like disk space shortages or system errors using canned scripts.",
    "Security features in IPM+ include real-time audits of CIS policies, blacklisting of unauthorized applications, and monitoring of firewall and antivirus health across all devices.",
    "The software provides component-level change management, alerting IT teams immediately if hardware parts like RAM or storage drives are removed or altered.",
    "IPM+ simplifies the management of distributed workforces by allowing hybrid agents to auto-switch between corporate networks and work-from-anywhere environments while remaining secure.",
    "The platform's remote remediation capabilities allow IT support teams to fix critical issues on thousands of devices at once without needing physical access to the hardware.",
    "Vigyanlabs is pioneering the development of India's first Green Micro Data Centers, which use renewable energy and patented cooling technologies to reduce energy consumption.",
    "The IPM+ Green Cloud offers a next-generation infrastructure with transparent, fixed pricing and a secure Virtual Private Cloud (VPC) environment for every enterprise client.",
    "Vigyanlabs' micro data centers achieve an unmatched Power Usage Effectiveness score of 1.2, which is significantly better than traditional large-scale hyperscale data centers.",
    "The IPM+ for Data Center solution provides a unified dashboard that monitors power, cooling, compute, storage, network, and virtualized containers in a single view.",
    "Vigyanlabs' cloud architecture is reimagined from the ground up as an 'Edge-First' sovereign cloud, reducing enterprise reliance on centralized, energy-intensive IT infrastructure providers.",
    "The Green Micro DC network is powered by solar energy and features no electrical switches in the building, using 100 percent AI-driven management for high availability.",
    "Vigyanlabs helps businesses accelerate AI adoption by providing managed cloud platforms that offer one-click deployment for Generative AI and other complex computational workloads.",
    "The IPM+ Private Cloud solution is a bespoke offering that allows businesses to host Vigyanlabs' cloud architecture on their own physical equipment and on-premises infrastructure.",
    "By utilizing patented DC optimization technologies, Vigyanlabs can reduce the energy footprint of traditional data centers by up to 50 percent while maintaining 99.9% uptime.",
    "The 'AI Energy Paradox' is a core principle at Vigyanlabs, suggesting that while AI consumes energy, it can also be the key to saving energy.",
    "Vigyanlabs offers Industry 4.0 IoT solutions that integrate smart sensors and gateways to monitor factory floors, office buildings, and complex industrial workflows in real-time.",
    "The Smart Building Solution uses IoT sensors to automate building operations, identify potential equipment failures, and track energy costs across lighting and climate control systems.",
    "Vigyanlabs provides the world's smallest 'gateway in a box,' which facilitates rapid sensor integration and remote monitoring for diverse industrial and commercial applications.",
    "The IoT workflow engine allows businesses to set complex rules for automated responses to environmental changes, such as adjusting cooling based on machine load or occupancy.",
    "Real-time energy audits provided by Vigyanlabs' IoT platform help factories identify 'energy leaks' and optimize production cycles to align with lower-cost energy periods.",
    "The company's Industry 4.0 solutions are designed for a low Total Cost of Ownership and a quick Return on Investment for manufacturing enterprises.",
    "Vigyanlabs' IoT stack includes secure data mirroring and synchronization to ensure that critical operational data is always available for analysis and decision-making.",
    "The platform supports various communication protocols to connect legacy machinery with modern cloud-based monitoring systems for a complete digital transformation of the factory.",
    "Integrated UPS monitoring through the IPM+ suite provides real-time reporting on backup power health and energy discharge rates during power outages in industrial sites.",
    "Vigyanlabs continues to innovate in the space of 'Net-Positive' green data centers, where the facility generates more renewable energy than it consumes for its operations.",
    "IPM+ Employee Productivity Management provides automated timesheets and analyzes application usage hours to help businesses understand workforce engagement and optimize daily operations.",
    "The productivity module uses AI-driven insights to monitor active versus idle time, helping companies maintain a healthy work culture while ensuring operational efficiency and compliance.",
    "Vigyanlabs' Patch Management solution automates the deployment of critical updates, ensuring that all endpoints are protected against the latest security vulnerabilities without manual intervention.",
    "The smart patching engine includes bandwidth throttling to ensure that large software updates do not disrupt network performance for users in bandwidth-constrained remote locations.",
    "Administrators can pause or schedule patching cycles based on real-time organizational needs, ensuring that updates are only applied during non-critical business hours for each department.",
    "Detailed patch progress reports provide IT managers with a clear view of which devices are up-to-date and which require immediate attention to remain secure.",
    "The productivity suite also features print management tools that track paper and cartridge usage, helping enterprises reduce printing costs and environmental waste significantly.",
    "IPM+ provides actionable intelligence dashboards that combine productivity data with hardware performance to identify correlations between device health and employee output.",
    "The system's 'AI Help Desk' uses automated diagnostics to resolve user issues faster, reducing the burden on IT support teams and increasing overall organizational uptime.",
    "Vigyanlabs' employee monitoring tools are designed with privacy and compliance in mind, focusing on high-level productivity trends rather than intrusive individual surveillance.",
    "Vigyanlabs' products are cybersecurity-compliant and VAPT certified, making them a trusted choice for sensitive environments such as defense, banking, and government infrastructure.",
    "The IPM+ Policy & Compliance Engine allows for the creation of dynamic groups, where different security policies are automatically applied based on the user's role or location.",
    "Security features include file and registry monitoring to detect unauthorized system changes that might indicate a malware infection or an attempted data breach.",
    "The solution monitors the health of existing endpoint security tools like antivirus and firewalls, alerting administrators if these critical defenses are disabled or out-of-date.",
    "Vulnerability scanning and reporting within IPM+ identify open ports and insecure services, providing a comprehensive risk assessment for the entire enterprise network.",
    "Vigyanlabs ensures defense-grade security for all its cloud and endpoint solutions, adhering to international standards like CIS benchmarks.",
    "The IPM+ platform provides non-intrusive security monitoring, ensuring that safety checks do not interfere with the speed or performance of the user's daily tasks.",
    "Real-time critical patching allows for the immediate deployment of emergency security fixes across the entire organization to mitigate zero-day threats within minutes of discovery.",
    "The centralized management console provides a secure, encrypted channel for all communication between the IPM+ agents and the cloud-based or on-premises server.",
    "Vigyanlabs' solutions help enterprises meet their ESG goals by providing auditable data on both energy savings and data security compliance.",
    "The IPM+ agent is written to be extremely efficient, often consuming less than 10MB of memory and under 3 percent of CPU on a standard business laptop.",
    "Vigyanlabs utilizes advanced AI algorithms for 'Hyper-Personalized' energy savings, adjusting the power profile of each individual machine based on specific user behavior patterns.",
    "The platform's architecture is designed for extreme scalability, proven to handle over 100,000 devices distributed across thousands of different geographical locations in a single deployment.",
    "Software-based energy metering in IPM+ eliminates the need for expensive hardware meters, providing a cost-effective way for companies to measure their IT energy consumption.",
    "The system includes hardware health management that monitors battery wear levels, fan performance, and operating temperatures to prevent unexpected hardware failures and data loss.",
    "Vigyanlabs' cloud and data center technologies utilize a software-defined approach, allowing for fully automated management of server resources and environmental controls.",
    "The IPM+ platform is built for deep insight, analyzing over 200 parameters per endpoint to provide a granular view of the organization's entire digital estate.",
    "Vigyanlabs' private cloud solution offers true elastic compute capabilities, allowing enterprises to scale their processing power up or down based on real-time demand.",
    "The system supports DIY reporting and dashboards, enabling users to create custom visualizations of their data without needing any specialized coding or technical skills.",
    "Vigyanlabs' technology has been validated by PassMark benchmarks, confirming that the software provides maximum efficiency with minimal impact on system performance or storage space.",
    "The State Bank of India, one of the world's largest banks, has successfully used IPM+ to save over 24.97 GWH of energy annually across its branch network.",
    "Vigyanlabs' solutions have helped sequestrate thousands of metric tons of CO2 emissions, contributing to the global fight against climate change through smarter IT management.",
    "Customers using Vigyanlabs' IPM+ have reported cost savings of over $450 million collectively, demonstrating the high financial return on sustainable technology investments.",
    "Vigyanlabs is listed as a top competitor in the global AI-powered cloud and endpoint management market, frequently cited alongside major international green technology firms.",
    "The company's products have been featured in several books and academic journals as prime case studies for successful sustainable innovation in the Indian tech ecosystem.",
    "Vigyanlabs continues to expand its reach, recently raising significant seed funding to further develop its green micro data center network and sovereign AI cloud.",
    "The partnership between Vigyanlabs and Microsoft has enabled many enterprises to transition their on-premises infrastructure to more energy-efficient, AI-optimized cloud environments.",
    "Organizations using IPM+ report a 100 percent satisfaction rate due to the software's ability to 'pay for itself' through rapid energy and operational cost savings.",
    "Vigyanlabs' technology is a key enabler for 'Green IT' departments, providing the tools necessary to measure, report, and reduce the environmental impact of computing.",
    "As businesses worldwide face increasing pressure to adopt sustainable practices, Vigyanlabs remains at the forefront of the industry with its patented, AI-driven green solutions."
]

### Load the model

In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print('Model loaded successfully')

Model loaded successfully


### Embedding generation

In [5]:
embeddings = model.encode(sentences, normalize_embeddings = True)

In [6]:
print(embeddings.shape)

(100, 384)


In [7]:
print(type(embeddings))

<class 'numpy.ndarray'>


In [8]:
print('Type of the embeddings is', type(embeddings))
print('Type of the element inside it is', type(embeddings[0][0]))

Type of the embeddings is <class 'numpy.ndarray'>
Type of the element inside it is <class 'numpy.float32'>


### Create and populate the FAISS index

In [20]:
# Obtain the dimension of the vector embedding
d = embeddings.shape[1]

# Create an index with inner product similarity
index = faiss.IndexFlatIP(d)

# Populate it
index.add(embeddings)

### Document upload functionality

In [21]:
def process_uploaded_files(uploader_value):
    """
    Takes uploader.value (tuple of dicts), extracts text, splits into sentences,
    embeds them, and returns a dict {filename: {"sentences": [...], "embeddings": [...]}}.
    """
    results = {}
    for fileinfo in uploader_value:
        name = fileinfo['name']
        content = bytes(fileinfo['content'])
        ext = name.lower().split('.')[-1]
        

        # --- Extract text ---
        if ext == 'txt':
            text = content.decode('utf-8', errors='ignore')

        elif ext == 'pdf':
            text = ""
            with pdfplumber.open(BytesIO(content)) as pdf:
                for page_num, page in enumerate(pdf.pages, start=1):
                    page_text = page.extract_text()
                    if page_text:
                        text += page_text + "\n"
                    print(f"Page {page_num} length: {len(page_text) if page_text else 0}")

        elif ext == 'docx':
            text = ""
            doc = docx.Document(BytesIO(content))
            for para_num, para in enumerate(doc.paragraphs, start=1):
                text += para.text + "\n"
                if para.text.strip():
                    print(f"Paragraph {para_num} length: {len(para.text)}")

        else:
            print(f"⚠️ Unsupported file type: {ext}")
            continue

        print(f"Total extracted text length: {len(text)} characters")
        print("Text preview:", text[:200], "...")

        # --- Split into sentences ---
        sentences = sent_tokenize(text)
        print(f"Number of sentences extracted: {len(sentences)}")

        # --- Embed sentences ---
        embeddings = model.encode(sentences, normalize_embeddings=True)
        print("Embeddings shape:", embeddings.shape)

        # --- Store results ---
        results[name] = {
            "sentences": sentences,
            "embeddings": embeddings
        }


    return results


### Add the data from the documents to our base dataset

In [22]:
def add_context(content):
    global sentences, embeddings, index

    print("Raw uploader content type:", type(content))

    processed = process_uploaded_files(content)
    print("Processed files:", list(processed.keys()))

    uploaded_sentences = []
    uploaded_embeddings = []

    for fname, data in processed.items():
        print(f"\nFile: {fname}")
        uploaded_sentences.extend(data["sentences"])
        uploaded_embeddings.extend(data["embeddings"])

    # Merge into global lists
    sentences.extend(uploaded_sentences)
    embeddings = np.vstack([embeddings, uploaded_embeddings])

    uploaded_embeddings = np.array(uploaded_embeddings).astype('float32')
    index.add(uploaded_embeddings)


### Question-Answering

In [27]:
# Define a messages list to store conversations
# Added a system message to guide the model's behavior

# Messages list
messages = [
    {'role': 'system', 'content': '''Use uploaded documents as context based on the knowledge base. Answer confidently to the point'''}
]

def ask_local_llm(query, messages):
    print("User query:", query)

    query_vec = model.encode([query], normalize_embeddings=True)
    print("Query embedding shape:", query_vec.shape)

    D, I = index.search(query_vec, k=5)
    print("Distances:", D)
    print("Indices:", I)

    retrieved = []
    for idx in I[0]:
        if idx < len(sentences):
            print("Retrieved sentence:", sentences[idx])
            retrieved.append(sentences[idx])
        else:
            print("⚠️ Index out of range:", idx)

    context = "\n".join(retrieved)

    if context.strip():
        user_msg = f"Context:\n{context}\n\nQuestion: {query}"
    else:
        user_msg = f"Question: {query}"

    messages.append({'role': 'user', 'content': user_msg})

    response = ollama.chat(
        model='llama3.2',
        messages=messages,
        options={'temperature': 0}
    )

    answer = response['message']['content']
    messages.append({'role': 'assistant', 'content': answer})

    return answer


### Chat Function

In [28]:
import ipywidgets as widgets
from IPython.display import display

# Create widgets once
uploader = widgets.FileUpload(accept='.txt,.pdf,.docx', multiple=True)
query_box = widgets.Text(description="Ask:")
ask_button = widgets.Button(description="Ask", button_style="primary")
output_area = widgets.Output()

# Display UI
display(uploader, query_box, ask_button, output_area)

# Handle file uploads
def on_upload_change(change):
    if uploader.value:
        with output_area:
            print("User uploaded a doc")
        add_context(uploader.value)   # same structure as before
        with output_area:
            print("Documents uploaded and indexed.\n")

uploader.observe(on_upload_change, names='value')

# Handle queries (only when Ask button is clicked)
def run_query(query):
    if not query.strip():
        return  # ignore empty queries

    # Reset messages to just system + current user
    local_messages = [messages[0], {'role': 'user', 'content': query}]

    answer = ask_local_llm(query, local_messages)
    with output_area:
        print(f"Assistant: {answer}\n")

# Bind only the button click
ask_button.on_click(lambda btn: run_query(query_box.value))


FileUpload(value=(), accept='.txt,.pdf,.docx', description='Upload', multiple=True)

Text(value='', description='Ask:')

Button(button_style='primary', description='Ask', style=ButtonStyle())

Output()

In [29]:
import os
os.getcwd()

'C:\\Users\\bhargav'